# **Prédictions des années de plantations des arbres de Grenoble**

Bienvenue dans le notebook prédiction des années de plantations des arbres de Grenoble. Les données utilisées sont des informations recueillies sur les arbres plantés à Grenoble de ... 

## **Objectifs :**
- Analyser les données des arbres de Grenoble
- Construire plusieurs modèles de machine learning pour prédire les années de plantation de ces  arbres de Grenoble.
- Faire une évaluation robuste grâce à la cross validation.
- Améliorer leurs performances grâce à la Grid search
- Comparer ces modèles entre elles 

### Étape 1 : **Importation des modules requise pour l'étude**

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)

### Étape 2 : **Chargement des données** 

In [2]:
df_arbres = pd.read_csv("data/arbres_grenoble_epsg4326.csv")
df_arbres.head()

,elem_point_id,code,nom,genre,genre_desc,categorie,categorie_desc,sous_categorie,sous_categorie_desc,code_parent,code_parent_desc,adr_secteur,bien_reference,genre_bota,espece,variete,stadededeveloppement,equipe,remarques,anneedeplantation,raisondeplantation,traitementchenilles,courrier,identifiantplu,typeimplantationplu,intituleprotectionplu,anneeabattage,essouchement,diametrearbre,causeabattage,collectivite,stationmetro,hauteurarbre,portarbre,structure,forme,typenature,geo_point_2d
0,25301,ESP30399,AR30399,VEG,VEGETATION,ESP01,Arbre,ESP187,Bois semi naturel,ESP761,Grande pelouse butte des pins,6,AR30399,Pinus,nigra,Austriaca,Arbre adulte,NaN,fourche à 7 m,1975.0,NaN,Moyenne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,"45.164519466065315,5.7364309029098335"
1,17909,ESP5185,AR5185,VEG,VEGETATION,ESP01,Arbre,ESP187,Bois semi naturel,ESP761,Grande pelouse butte des pins,6,AR5185,Pinus,nigra,Austriaca,Arbre adulte,NaN,dominé,1975.0,NaN,Moyenne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,"45.16450876797978,5.736451624755175"
2,28960,ESP16027,AR16027,VEG,VEGETATION,ESP01,Arbre,ESP187,Bois semi naturel,ESP761,Grande pelouse butte des pins,6,AR16027,Pinus,nigra,Austriaca,Arbre adulte,NaN,NaN,1975.0,NaN,Moyenne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,"45.164556703715526,5.736307688353201"
3,14246,ESP17702,AR17702,VEG,VEGETATION,ESP01,Arbre,ESP187,Bois semi naturel,ESP761,Grande pelouse butte des pins,6,AR17702,Pinus,nigra,Austriaca,Arbre adulte,NaN,NaN,1975.0,NaN,Moyenne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,"45.164533444541846,5.736391508663688"
4,45677,ESP41299,AR41299,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP137,Sq Waldeck Rousseau,1,AR41299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"45.18330787379752,5.709045164969469"


### Étape 3 : **Prétraitement des données**

In [3]:
df_arbres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31895 entries, 0 to 31894
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   elem_point_id          31895 non-null  int64  
 1   code                   31895 non-null  object 
 2   nom                    31895 non-null  object 
 3   genre                  31895 non-null  object 
 4   genre_desc             31895 non-null  object 
 5   categorie              31895 non-null  object 
 6   categorie_desc         31895 non-null  object 
 7   sous_categorie         31895 non-null  object 
 8   sous_categorie_desc    31895 non-null  object 
 9   code_parent            31895 non-null  object 
 10  code_parent_desc       31895 non-null  object 
 11  adr_secteur            31895 non-null  int64  
 12  bien_reference         31895 non-null  object 
 13  genre_bota             31079 non-null  object 
 14  espece                 29415 non-null  object 
 15  va

##### *Variables chaines de caracteres: 28 colonnes*
##### *Variables Numerique: 10 colonnes dont 6 ne possedant aucunes informations*
##### *Total: 38 colonnes*


##### Suppression de colonnes non indispensables

Toutes les colonnnes dont le nombre total de valeurs manquantes sont  au dessus d'un certain seuil

On supprime toutes les colonnes ne possédant aucune informations ou possédant très peu d'informations

In [4]:
def high_nan_Columns_list(df, seuil):
    
    """Retourne une liste contenant des une proportion de valeur manquante superieur a un seuil."""
    
    return [elm for elm in df.columns if (df[elm].isna().sum()/df.shape[0])*100 >= seuil] 


list90nan = high_nan_Columns_list(df_arbres, 70)

# suppresion des  colonnes de plus de 90% de valeurs manquantes

df_arbres = df_arbres.drop(list90nan, axis = 1)

Maintenant Examinons le nombre valeurs uniques par colonnes

In [5]:
df_arbres.nunique()

elem_point_id           31895
code                    31895
nom                     31895
genre                       1
genre_desc                  1
categorie                   1
categorie_desc              1
sous_categorie              4
sous_categorie_desc         4
code_parent              1506
code_parent_desc         1506
adr_secteur                11
bien_reference          31895
genre_bota                120
espece                    292
stadededeveloppement        3
anneedeplantation          68
collectivite                2
hauteurarbre                3
portarbre                   7
structure                   4
typenature                  7
geo_point_2d            31895
dtype: int64

on supprime toute les colonnes qui n'ont qu'une seule valeurs unique, car ca ne nous apporte pas reellement d'information. 

In [6]:
df_arbres=df_arbres[df_arbres.columns[df_arbres.nunique()>1]]

In [7]:
print('Nombre restante de colonnes:',df_arbres.nunique().count())

Nombre restante de colonnes: 19


Observons maintenant les valeurs que contiennent chaque colonne. 

In [8]:
df_arbres

,elem_point_id,code,nom,sous_categorie,sous_categorie_desc,code_parent,code_parent_desc,adr_secteur,bien_reference,genre_bota,espece,stadededeveloppement,anneedeplantation,collectivite,hauteurarbre,portarbre,structure,typenature,geo_point_2d
0,25301,ESP30399,AR30399,ESP187,Bois semi naturel,ESP761,Grande pelouse butte des pins,6,AR30399,Pinus,nigra,Arbre adulte,1975.0,Ville de Grenoble,NaN,NaN,Ville de Grenoble,NaN,"45.164519466065315,5.7364309029098335"
1,17909,ESP5185,AR5185,ESP187,Bois semi naturel,ESP761,Grande pelouse butte des pins,6,AR5185,Pinus,nigra,Arbre adulte,1975.0,Ville de Grenoble,NaN,NaN,Ville de Grenoble,NaN,"45.16450876797978,5.736451624755175"
2,28960,ESP16027,AR16027,ESP187,Bois semi naturel,ESP761,Grande pelouse butte des pins,6,AR16027,Pinus,nigra,Arbre adulte,1975.0,Ville de Grenoble,NaN,NaN,Ville de Grenoble,NaN,"45.164556703715526,5.736307688353201"
3,14246,ESP17702,AR17702,ESP187,Bois semi naturel,ESP761,Grande pelouse butte des pins,6,AR17702,Pinus,nigra,Arbre adulte,1975.0,Ville de Grenoble,NaN,NaN,Ville de Grenoble,NaN,"45.164533444541846,5.736391508663688"
4,45677,ESP41299,AR41299,ESP174,Arbre d'espaces ouverts,ESP137,Sq Waldeck Rousseau,1,AR41299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"45.18330787379752,5.709045164969469"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31890,22630,ESP28052,AR28052,ESP151,Arbre de voirie,ESP1287,Bd J Vallier voie tram pair,1,AR28052,Liriodendron,tulipifera,Arbre jeune,2005.0,Grenoble Alpes Métropole,de 10 m à 20 m,Libre,Métropole de Grenoble,Libre,"45.180145884199455,5.708644349254183"
31891,22634,ESP28053,AR28053,ESP151,Arbre de voirie,ESP1287,Bd J Vallier voie tram pair,1,AR28053,Betula,verrucosa,Arbre jeune,2005.0,Grenoble Alpes Métropole,Moins de 10 m,Libre,Métropole de Grenoble,Libre,"45.18022983714377,5.712917076957194"
31892,22644,ESP28055,AR28055,ESP151,Arbre de voirie,ESP195,R Ampère ang Lindbergh,1,AR28055,Liquidambar,styraciflua,Arbre adulte,1989.0,Grenoble Alpes Métropole,NaN,NaN,Métropole de Grenoble,NaN,"45.18314214954124,5.705678144597991"
31893,22646,ESP28056,AR28056,ESP151,Arbre de voirie,ESP1288,R DrGreffier,1,AR28056,Acer,negundo,Arbre adulte,1996.0,Grenoble Alpes Métropole,Moins de 10 m,Semi-libre,Métropole de Grenoble,Semi-libre,"45.18164036492663,5.705245792013283"


In [9]:
df_arbres.nunique()

elem_point_id           31895
code                    31895
nom                     31895
sous_categorie              4
sous_categorie_desc         4
code_parent              1506
code_parent_desc         1506
adr_secteur                11
bien_reference          31895
genre_bota                120
espece                    292
stadededeveloppement        3
anneedeplantation          68
collectivite                2
hauteurarbre                3
portarbre                   7
structure                   4
typenature                  7
geo_point_2d            31895
dtype: int64

-Les colonnes code, nom,bien_reference sont des variables categorielles de 31895 classes. Ce qui veut dire que ces categories sont caracteristiques de chaque arbres, pas un groupe d'arbres. Nous allons alors les supprimees. 

-La colonne elem_point_id est une colonne categorielle mais numerique. Elle represente l'identite numerique de l'arbre, et est probablement associe a chaque arbre par ordre croissant. cette coloenne est a conserve.


-Les colonnes Portarbre, collectivite, remarques, code_parent_desc (description du code parent), sous_categorie_desc(description de la sous categorie), raisondeplantation, typeimplantationplu, typenature, forme, sont a supprime:


In [11]:
colonne_a_supprimer = ['portarbre','typenature','collectivite','code_parent_desc','sous_categorie_desc','typenature', 'code','nom','bien_reference']

df_arbres = df_arbres.drop(colonne_a_supprimer, axis = 1)

In [12]:
df_arbres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31895 entries, 0 to 31894
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   elem_point_id         31895 non-null  int64  
 1   sous_categorie        31895 non-null  object 
 2   code_parent           31895 non-null  object 
 3   adr_secteur           31895 non-null  int64  
 4   genre_bota            31079 non-null  object 
 5   espece                29415 non-null  object 
 6   stadededeveloppement  27645 non-null  object 
 7   anneedeplantation     29756 non-null  float64
 8   hauteurarbre          20199 non-null  object 
 9   structure             31080 non-null  object 
 10  geo_point_2d          31895 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 2.7+ MB


On compte a nouveau la quantite 

In [13]:
(df_arbres.isna().sum()/df_arbres.shape[0])*100

elem_point_id            0.000000
sous_categorie           0.000000
code_parent              0.000000
adr_secteur              0.000000
genre_bota               2.558395
espece                   7.775513
stadededeveloppement    13.324973
anneedeplantation        6.706380
hauteurarbre            36.670325
structure                2.555259
geo_point_2d             0.000000
dtype: float64

 Transformons la colonne geo_point_2d en deux colonnes, l'une pour la longitude et l'autre pour la latitude.

In [14]:
def longitude(x):
    return x.split(',')[1]
def latitude(x):
    return x.split(',')[0]

df_arbres['longitude'] = df_arbres['geo_point_2d'].apply(longitude).apply(pd.to_numeric,errors = 'coerce')
df_arbres['latitude'] = df_arbres['geo_point_2d'].apply(latitude).apply(pd.to_numeric,errors = 'coerce')

#supression de la colonne geo_point_2d
df_arbres = df_arbres.drop(['geo_point_2d'],axis = 1)

In [15]:
df_arbres

,elem_point_id,sous_categorie,code_parent,adr_secteur,genre_bota,espece,stadededeveloppement,anneedeplantation,hauteurarbre,structure,longitude,latitude
0,25301,ESP187,ESP761,6,Pinus,nigra,Arbre adulte,1975.0,NaN,Ville de Grenoble,5.736431,45.164519
1,17909,ESP187,ESP761,6,Pinus,nigra,Arbre adulte,1975.0,NaN,Ville de Grenoble,5.736452,45.164509
2,28960,ESP187,ESP761,6,Pinus,nigra,Arbre adulte,1975.0,NaN,Ville de Grenoble,5.736308,45.164557
3,14246,ESP187,ESP761,6,Pinus,nigra,Arbre adulte,1975.0,NaN,Ville de Grenoble,5.736392,45.164533
4,45677,ESP174,ESP137,1,NaN,NaN,NaN,NaN,NaN,NaN,5.709045,45.183308
...,...,...,...,...,...,...,...,...,...,...,...,...
31890,22630,ESP151,ESP1287,1,Liriodendron,tulipifera,Arbre jeune,2005.0,de 10 m à 20 m,Métropole de Grenoble,5.708644,45.180146
31891,22634,ESP151,ESP1287,1,Betula,verrucosa,Arbre jeune,2005.0,Moins de 10 m,Métropole de Grenoble,5.712917,45.180230
31892,22644,ESP151,ESP195,1,Liquidambar,styraciflua,Arbre adulte,1989.0,NaN,Métropole de Grenoble,5.705678,45.183142
31893,22646,ESP151,ESP1288,1,Acer,negundo,Arbre adulte,1996.0,Moins de 10 m,Métropole de Grenoble,5.705246,45.181640


#### Separation des donnees

notre objectif est de predire les annees de planttion des arbres qui n'en ont pas. pour cela nous allons enregistrer dans deux fichier differents les arbres qui en ont de ceux qui n'ent on pas.

In [16]:
# fichier des arbres sans annees de plantation
df_arbres[df_arbres['anneedeplantation'].isna() ==True].to_pickle("/home/moi/Téléchargements/Documents (2)/Machine_learning/arbres_sans_annees.pkl", protocol=4)

In [17]:
# fichier des arbres avec annees de plantation 
df_arbres[df_arbres['anneedeplantation'].isna() ==False].to_pickle("/home/moi/Téléchargements/Documents (2)/Machine_learning/arbres_avec_annees.pkl",protocol =4 )

###